# Mesure de chevauchement et recalage d'images

Ce chapitre explore les méthodes pour quantifier le chevauchement entre deux images ou régions. Ces techniques sont essentielles en imagerie médicale pour :
-   Évaluer la qualité du recalage (registration) d'images
-   Comparer des segmentations automatiques à des annotations manuelles
-   Mesurer la similarité entre structures anatomiques

Nous comparerons différentes métriques de similarité :
-   **Dice** : Mesure standard pour la similarité de régions
-   **Weighted Dice** : Version pondérée pour les données continues
-   **Corrélation** : Mesure la dépendance linéaire entre les intensités
-   **Distance euclidienne** : Mesure la distance entre les centres de masse

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from utils import generate_square  # Fonction utilitaire pour générer des carrés

---

## 1. Génération d'images synthétiques avec distribution gaussienne

Pour tester nos métriques, nous créons des images synthétiques avec des carrés positionnés aléatoirement :
-   **Distribution gaussienne** : Centres et tailles suivent une loi normale
-   **Chevauchement progressif** : Les carrés s'accumulent, créant des régions de forte densité
-   **`np.random.seed()`** : Assure la reproductibilité

Cette approche simule des cas réalistes où les régions peuvent se chevaucher partiellement.

In [ ]:
# Initialize 1000 squares with center (x,y) and size (x,y) following a gaussian distribution

np.random.seed(1234)
shape = (500, 500)
arr_smooth = np.zeros(shape)
center = np.random.normal(200, 20, size=2000).reshape((1000, 2))
size = np.random.normal(50, 20, size=2000).reshape((1000, 2))

for i in range(1000):
    arr_smooth += generate_square(shape, center[i], size[i])

plt.imshow(arr_smooth)
plt.show()

---

## 2. Génération d'images avec distribution uniforme

Pour contraster avec la distribution gaussienne, nous créons une distribution uniforme :
-   **Distribution uniforme** : Positions et tailles aléatoires sur toute l'image
-   **Moins de chevauchement** : Les carrés sont plus dispersés
-   **Seed différent** : Garantit une distribution indépendante

Cette configuration sert de référence pour comparer les métriques de chevauchement.

In [ ]:
# Initialize 1000 squares with center (x,y) and size (x,y) following an uniform distribution
np.random.seed(1066)
arr = np.zeros(shape)
center = np.random.random(400).reshape((200, 2)) * arr.shape[0]
size = np.random.random(400).reshape((200, 2)) * arr.shape[0] // 5

for i in range(200):
    arr += generate_square(shape, center[i], size[i])

plt.imshow(arr)
plt.show()

---

## 3. Calcul de métriques de chevauchement (Dice, Weighted Dice, Corrélation)

Nous testons trois métriques pour quantifier le chevauchement :

**1. Coefficient de Dice** :
$$Dice = \frac{2|A \cap B|}{|A| + |B|}$$
-   Varie de 0 (aucun chevauchement) à 1 (chevauchement parfait)
-   Standard en segmentation médicale

**2. Weighted Dice** :
-   Extension du Dice pour les valeurs continues (non binaires)
-   Pondère par l'intensité des pixels

**3. Corrélation** :
$$r = \frac{cov(A, B)}{\sigma_A \sigma_B}$$
-   Mesure la dépendance linéaire
-   Sensible à la distribution des intensités

**Stratégie** : Générer des carrés un par un, calculer les métriques à chaque ajout

In [ ]:
# One by one, generate the uniform distribution of square and compute their overlap using 3 metrics
# Dice, Weigthed-Dice and Correlation, those with good (enough) score should be kept

np.random.seed(1066)
arr = np.zeros(shape)
center = np.random.random(400).reshape((200, 2)) * arr.shape[0]
size = np.random.random(400).reshape((200, 2)) * arr.shape[0] // 5

def compute_dice_voxel(density_1, density_2):
    overlap_idx = np.nonzero(density_1 * density_2)
    numerator = 2 * len(overlap_idx[0])
    denominator = np.count_nonzero(density_1) + np.count_nonzero(density_2)

    if denominator > 0:
        dice = numerator / float(denominator)
    else:
        dice = np.nan

    overlap_1 = density_1[overlap_idx]
    overlap_2 = density_2[overlap_idx]
    w_dice = np.sum(overlap_1) + np.sum(overlap_2)
    denominator = np.sum(density_1) + np.sum(density_2)
    if denominator > 0:
        w_dice /= denominator
    else:
        w_dice = np.nan

    return dice, w_dice


def compute_correlation(density_1, density_2):
    indices = np.where(density_1 + density_2 > 0)
    if np.array_equal(density_1, density_2):
        density_correlation = 1
    elif (np.sum(density_1) > 0 and np.sum(density_2) > 0) \
            and np.count_nonzero(density_1 * density_2):
        density_correlation = np.corrcoef(density_1[indices],
                                          density_2[indices])[0, 1]
    else:
        density_correlation = 0

    return max(0, density_correlation)

arr = np.zeros(shape)
for i in range(200):
    curr = generate_square(shape, center[i], size[i])
    dice, w_dice = compute_dice_voxel(arr_smooth, curr)
    corr = compute_correlation(arr_smooth, curr)
    # This value of 0.1 control how strict the search is
    if dice + w_dice + corr > 0.1:
        arr += curr

plt.imshow(arr)
plt.show()

---

## 4. Distance euclidienne entre les barycentres

Une approche alternative consiste à comparer les centres de masse (barycentres) :
-   **`np.where()`** : Extraction des positions de tous les pixels non nuls
-   **`np.average(weights=...)`** : Calcul du barycentre pondéré par l'intensité
-   **Distance euclidienne** : $d = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$

**Avantages** :
-   Très rapide à calculer
-   Intuitive (distance physique entre les centres)
-   Robuste aux variations de forme

**Limitations** :
-   Ne capture pas la forme ou l'étendue du chevauchement
-   Peut être trompeuse si les distributions sont très asymétriques

In [ ]:
# One by one, generate the uniform distribution of square and compute their overlap using a distance to barrycenter

np.random.seed(1066)
arr = np.zeros(shape)
center = np.random.random(400).reshape((200, 2)) * arr.shape[0]
size = np.random.random(400).reshape((200, 2)) * arr.shape[0] // 5

pos_smooth = np.where(arr_smooth)
val_smooth = arr_smooth[pos_smooth]
center_smooth = np.average(pos_smooth, axis=1, weights=val_smooth)
for i in range(200):
    curr = generate_square(shape, center[i], size[i])
    pos = np.where(curr)
    if len(pos[0]) > 1:
        pos = np.average(pos, axis=1)
        # This value represent a distance in pixel
        if np.linalg.norm(pos - center_smooth) < 100:
            arr += curr

plt.imshow(arr)
plt.show()